# Instalação

In [14]:
%pip install pyspark
%pip install pandas
%pip install scikit-learn
%pip install numpy


Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


# Importação

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


# Dataframe

In [5]:
# Inicializa uma sessão Spark
spark = SparkSession.builder.appName('modelo-de-recomendacao-baseado-em-conteudo').getOrCreate()

# Carrega os dados em um DataFrame Spark
df = spark.read.csv('./data/processed/base-de-dados-final.csv', inferSchema=True, header=True)

df = df.drop('proponente_id', 'avaliador_id', 'avaliacao')

df = df.dropDuplicates(['projeto_id'])

colunas_para_vetorizar = [coluna for coluna in df.columns if coluna != 'projeto_id']

# Vetoriza as colunas do DataFrame
assembler = VectorAssembler(inputCols=colunas_para_vetorizar, outputCol="vetor")
df_vetorizado = assembler.transform(df)
df_vetorizado = df_vetorizado.drop(*[coluna for coluna in df.columns if coluna in colunas_para_vetorizar])

# Converte o df_vetorizado do Spark para Pandas
df = df_vetorizado.toPandas()

# Modelo

In [6]:
# Converte a coluna 'vetor' para uma matriz NumPy
vetor = np.stack(df['vetor'].apply(lambda x: x.toArray()))

# Calcula a similaridade do cosseno entre cada item
similaridade_cosseno = cosine_similarity(vetor, vetor)

# Cria um mapeamento reverso dos índices e títulos do filme
indices = pd.Series(df.index, index=df['projeto_id']).drop_duplicates()

# Resultados

In [14]:
def procurar_projetos_similares(projeto_id):
    # Obtêm as pontuações de similaridade de todos os projetos com o projeto especificado
    projetos = list(enumerate(similaridade_cosseno[projeto_id - 1]))

    projetos.pop(projeto_id - 1)
    
    # Classifica os projetos com base nas pontuações de similaridade
    projetos = sorted(projetos, key=lambda x: x[1], reverse=True)

    # Obtêm os ids dos projeto
    projetos = [{"id": projeto[0] + 1, "similaridade": projeto[1]} for projeto in projetos]

    return projetos


procurar_projetos_similares(25)

[{'id': 690, 'similaridade': 1.0000000000000002},
 {'id': 911, 'similaridade': 1.0000000000000002},
 {'id': 22, 'similaridade': 0.8333333333333336},
 {'id': 109, 'similaridade': 0.8333333333333336},
 {'id': 120, 'similaridade': 0.8333333333333336},
 {'id': 339, 'similaridade': 0.8333333333333336},
 {'id': 388, 'similaridade': 0.8333333333333336},
 {'id': 415, 'similaridade': 0.8333333333333336},
 {'id': 453, 'similaridade': 0.8333333333333336},
 {'id': 501, 'similaridade': 0.8333333333333336},
 {'id': 553, 'similaridade': 0.8333333333333336},
 {'id': 585, 'similaridade': 0.8333333333333336},
 {'id': 741, 'similaridade': 0.8333333333333336},
 {'id': 879, 'similaridade': 0.8333333333333336},
 {'id': 964, 'similaridade': 0.8333333333333336},
 {'id': 965, 'similaridade': 0.8333333333333336},
 {'id': 1055, 'similaridade': 0.8333333333333336},
 {'id': 1127, 'similaridade': 0.8333333333333336},
 {'id': 1, 'similaridade': 0.6666666666666669},
 {'id': 6, 'similaridade': 0.6666666666666669},
 {'